In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
uaa_teams = ['WashU', 'Washington', 'Univ. of Chicago', 'Chicago', 'New York U', 'NYU', 'Emory', 'Rochester', 'Brandeis', 'Case Western Reserve', 'CWRU', 'Carnegie Mellon']

mbb_urls = {"NYU" : "https://gonyuathletics.com/sports/mens-basketball/stats/2022-23",
            "WashU" : "https://washubears.com/sports/mens-basketball/stats?path=mbball",
            "Brandeis" : "https://www.brandeisjudges.com/sports/mbkb/2022-23/teams/brandeis?view=gamelog",
            "Case Western" : "https://athletics.case.edu/sports/mbkb/2022-23/teams/casewesternreserve?view=gamelog",
            "Carnegie Mellon" : "https://athletics.cmu.edu/sports/mbkb/2022-23/teams/carnegiemellon?view=gamelog",
            "Emory" : "https://www.emoryathletics.com/sports/mbkb/2022-23/schedule",
            "Rochester" : "https://uofrathletics.com/sports/mens-basketball/stats/2022-23"}

wbb_urls = [u.replace("mbkb", "wbkb").replace("mens", "womens") for u in list(mbb_urls.values())]
wbb_urls = dict(map(lambda i, j : (i, j), list(mbb_urls.keys()), wbb_urls))

In [3]:
def str_list(string, lst):
    for i in lst:
        if string in i:
            return True
    return False

def fg_helper(splits):
    makes = sum([int(i.split("-")[0]) for i in splits])
    attempts = sum([int(i.split("-")[1]) for i in splits])
    
    return str(makes) + "-" + str(attempts)

def totals_helper(df):
    poss = (0.96*(sum(df['FGA'].astype(int)) + 
                  sum(df['TO'].astype(int)) + 
                  0.44*sum(df['FTA'].astype(int))) - 
            sum(df['OREB'].astype(int))) / len(df)
    opp_poss = (0.96*(sum(df['OPP FGA'].astype(int)) + 
                      sum(df['OPP TO'].astype(int)) + 
                      0.44*sum(df['OPP FTA'].astype(int))) - 
                sum(df['OPP OREB'].astype(int))) / len(df)
    ppp = round(sum(df['PTS'].astype(int)) / (poss * len(df)), 2)
    opp_ppp = round(sum(df['OPP PTS'].astype(int)) / (opp_poss * len(df)), 2)
    fgs = fg_helper( list(df['FGM/A']) )
    threes = fg_helper( list(df['3FG/A']) )
    fts = fg_helper( list(df['FTM/A']) )
    opp_fgs = fg_helper( list(df['OPP FGM/A']) )
    opp_threes = fg_helper( list(df['OPP 3PM/A']) )
    opp_fts = fg_helper( list(df['OPP FTM/A']) )
    orb = 100 * round(sum(df['OREB'].astype(int)) / (sum(df['OREB'].astype(int)) + sum(df['OPP DREB'].astype(int))), 3)
    drb = 100 * round(sum(df['DREB'].astype(int)) / (sum(df['DREB'].astype(int)) + sum(df['OPP OREB'].astype(int))), 3)
    reb = 100 * round(sum(df['REB'].astype(int)) / (sum(df['REB'].astype(int)) + sum(df['OPP REB'].astype(int))), 3)
    to = 100 * round(sum(df['TO'].astype(int)) / (poss * len(df)), 3)
    opp_to = 100 * round(sum(df['OPP TO'].astype(int)) / (opp_poss * len(df)), 3)

    return ["Total", "", sum(df['Margin']), 
            round(poss, 1), ppp, opp_ppp,
            fgs, threes, fts, opp_fgs, opp_threes, opp_fts,
            sum(df['REB'].astype(int)), sum(df['OPP REB'].astype(int)), 
            round(orb, 1), round(drb, 1), round(reb, 1), 
            round(to, 1), round(opp_to, 1)]

In [4]:
def game_by_game_df(driver, soup, school):
    table = soup.find_all('div', {"class" : "sidearm-table-overflow-on-x-large"})
    team = [t for t in table if "Team" in t.find("caption").text][0]
    opponents = [t for t in table if "Opponents" in t.find("caption").text][0]

    t_columns = [i.text for i in team.find("tr") if i.text != "\n"]
    t_entries = [[i.text for i in row.find_all("td")] for row in team.find_all("tr")[1:]]
    o_columns = [i.text for i in opponents.find("tr") if i.text != "\n"]
    o_entries = [[i.text for i in row.find_all("td")] for row in opponents.find_all("tr")[1:]]

    team_df = pd.DataFrame(t_entries[:-2], columns=t_columns)
    opponents_df = pd.DataFrame(o_entries[:-2], columns=o_columns)
    game_log = team_df.join(opponents_df, rsuffix=" OPP")

    game_log['Margin'] = (game_log['Score'].apply(lambda i : int(i.split("-")[0])) - 
                     game_log['Score'].apply(lambda i : int(i.split("-")[1])))
    game_log['FGM'] = game_log['FGM/A'].apply(lambda i : int(i.split("-")[0]))
    game_log['FGA'] = game_log['FGM/A'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP FGA'] = game_log['FGM/A OPP'].apply(lambda i : int(i.split("-")[1]))
    game_log['FTM'] = game_log['FTM/A'].apply(lambda i : int(i.split("-")[0]))
    game_log['FTA'] = game_log['FTM/A'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP FTM'] = game_log['FTM/A OPP'].apply(lambda i : int(i.split("-")[0]))
    game_log['OPP FTA'] = game_log['FTM/A OPP'].apply(lambda i : int(i.split("-")[1]))
    game_log['OREB'] = game_log['OFF']
    game_log['DREB'] = game_log['DEF']
    game_log['OPP OREB'] = game_log['OFF OPP']
    game_log['OPP DREB'] = game_log['DEF OPP']
    game_log['OPP TO'] = game_log['TO OPP']
    game_log['POSS'] = 0.96*(game_log['FGA'] + 
                        game_log['TO'].astype(int) + 
                        0.44*game_log['FTA'] - game_log['OFF'].astype(int))
    game_log['OPP POSS'] = 0.96*(game_log['OPP FGA'] + 
                            game_log['TO OPP'].astype(int) + 
                            0.44*game_log['OPP FTA'] - game_log['OFF OPP'].astype(int))
    game_log['FTM/A'] = game_log['FTM'].astype(str) + "-" + game_log['FTA'].astype(str)
    game_log['OPP FTM/A'] = game_log['OPP FTM'].astype(str) + "-" + game_log['OPP FTA'].astype(str)
    game_log['PPP'] = round(game_log['PTS'].astype(int) / game_log['POSS'], 2)
    game_log['OPP PTS'] = game_log['PTS OPP'].astype(int)
    game_log['OPP PPP'] = round(game_log['OPP PTS'].astype(int) / game_log['OPP POSS'], 2)
    game_log['REB'] = game_log['TOT']
    game_log['OPP REB'] = game_log['TOT OPP']
    game_log['ORB%'] = 100 * round(game_log['OFF'].astype(int) / 
                              (game_log['OFF'].astype(int) + game_log['DEF OPP'].astype(int)), 3)
    game_log['DRB%'] = 100 * round(game_log['DEF'].astype(int) / 
                              (game_log['DEF'].astype(int) + game_log['OFF OPP'].astype(int)), 3)
    game_log['REB%'] = 100 * round(game_log['TOT'].astype(int) / 
                              (game_log['TOT'].astype(int) + game_log['TOT OPP'].astype(int)), 3)
    game_log['TO%'] = 100 * round(game_log['TO'].astype(int) / game_log['POSS'], 3)
    game_log['OPP TO%'] = 100 * round(game_log['TO OPP'].astype(int) / game_log['OPP POSS'], 3)
    game_log['POSS'] = game_log['POSS'].round(1)
    game_log['OPP POSS'] = game_log['OPP POSS'].round(1)
    
    game_log['OPP FGM/A'] = game_log['FGM/A OPP']
    game_log['OPP 3PM/A'] = game_log['3FG/A OPP']
    
    totals_row = totals_helper(game_log)
    uaa_totals = totals_helper(game_log[(game_log.Opponent.apply(lambda o : 
                                                                 len([t for t in uaa_teams if t in o]) > 0))])
    
    game_log = game_log[['Opponent', 'W/L', 'Margin', 
                         'POSS', 'PPP', 'OPP PPP',
                         'FGM/A', '3FG/A', 'FTM/A', 'OPP FGM/A', 'OPP 3PM/A', 'OPP FTM/A',
                         'REB', 'OPP REB', 'ORB%', 'DRB%', 'REB%', 
                         'TO%', 'OPP TO%']]
    
    game_log.loc[len(game_log)] = uaa_totals
    game_log.loc[len(game_log)] = totals_row
    game_log.iloc[-2, game_log.columns.get_loc('Opponent')] = "UAA Total"
    game_log['Team'] = school

    return game_log

In [5]:
def opp_helper(soup, school, opp=True):
    if school == "Case Western":
        school = "Case Western Reserve"
        
    if opp:
        opponent = [t for t in soup.find_all("div", {"class" : "player-stats"})[0].find_all("table") 
                    if t.find("h2").text != school][0]
    else:
        opponent = [t for t in soup.find_all("div", {"class" : "player-stats"})[0].find_all("table") 
                    if t.find("h2").text == school][0]

    columns = [[th.text for th in tr.find_all("th")] for tr in opponent.find_all("tr")][0][2:]
    entries = [i.text.strip() for i in opponent.find("tr", {"class" : "totals"}).find_all("td")][1:]
    
    return pd.DataFrame([entries], columns=columns)

def game_log_df(driver, soup, school):
    if school == "Carnegie Mellon":
        columns = [th.text for th in soup.find(id="gamelog").find("tr").find_all("th")]
        rows = soup.find(id="gamelog").find_all("tr")[1:]
        entries = [[td.text.strip().replace("\t", "") for td in tr.find_all("td")] for tr in rows]
    else:
        columns = [i.text for i in soup.find_all("div", {"class" : "stats-box full clearfix"})[0].find_all("th")]
        rows = soup.find_all("div", {"class" : "stats-box full clearfix"})[0].find_all("tr")
        entries = [[td.text.strip().replace("\t", "") for td in r.find_all("td")] for r in rows][1:]
        
        if school == "Case Western":
            rows = [r for r in rows if not str_list("Buffalo State", [i.text for i in r.find_all("td")])]

    game_log = pd.DataFrame(entries, columns=columns)
    game_log = game_log[~(game_log.Score == "")]
    game_log = game_log.reset_index(drop=True)

    game_urls = [urls[school].split("/teams/")[0] + 
                 r.find("a")['href'].split("..")[1] 
                 for r in rows[0:game_log.shape[0] + 1] if r.find_all("a") != []]
    
    game_soups = []
    for url in game_urls:
        driver.get(url)
        game_soups.append( BeautifulSoup(driver.page_source, 'html.parser') )

    opponent_df = pd.concat([opp_helper(s, school) for s in game_soups])
    opponent_df = opponent_df.reset_index(drop=True)
    opponent_df['Opponent'] = game_log['Opponent']

    game_log = game_log.join(opponent_df, rsuffix=" OPP")

    game_log['W/L'] = game_log['Score'].apply(lambda i : i.split(",")[0])
    game_log['Margin'] = game_log.Score.apply(lambda i : eval(i.split(",")[1]))
    game_log['FGA'] = game_log['fg'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP FGM'] = game_log['FGM-A'].apply(lambda i : int(i.split("-")[0]))
    game_log['OPP FGA'] = game_log['FGM-A'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP 3PM'] = game_log['3PM-A'].apply(lambda i : int(i.split("-")[0]))
    game_log['OPP 3PA'] = game_log['3PM-A'].apply(lambda i : int(i.split("-")[1]))
    game_log['FTM'] = game_log['ft'].apply(lambda i : int(i.split("-")[0]))
    game_log['FTA'] = game_log['ft'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP FTM'] = game_log['FTM-A'].apply(lambda i : int(i.split("-")[0]))
    game_log['OPP FTA'] = game_log['FTM-A'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP OREB'] = game_log['OREB']
    game_log['OPP DREB'] = game_log['DREB']
    game_log['OREB'] = game_log['off']
    game_log['DREB'] = game_log['def']
    game_log['OPP TO'] = game_log['TO']
    game_log['TO'] = game_log['to']
    game_log['OPP PTS'] = game_log['PTS'].astype(int)
    game_log['PTS'] = game_log['pts']
    game_log['POSS'] = 0.96*(game_log['FGA'] + 
                        game_log['to'].astype(int) + 
                        0.44*game_log['FTA'] - game_log['off'].astype(int))
    game_log['OPP POSS'] = 0.96*(game_log['OPP FGA'] + 
                                 game_log['TO'].astype(int) + 
                                 0.44*game_log['OPP FTA'] - game_log['OREB'].astype(int))
    game_log['PPP'] = round(game_log['pts'].astype(int) / game_log['POSS'], 2)
    game_log['OPP PPP'] = round(game_log['OPP PTS'].astype(int) / game_log['OPP POSS'], 2)
    game_log['FGM/A'] = game_log['fg']
    game_log['3FG/A'] = game_log['3pt']
    game_log['OPP FGM/A'] = game_log['OPP FGM'].astype(str) + "-" + game_log['OPP FGA'].astype(str)
    game_log['OPP 3PM/A'] = game_log['OPP 3PM'].astype(str) + "-" + game_log['OPP 3PA'].astype(str)
    game_log['FTM/A'] = game_log['FTM'].astype(str) + "-" + game_log['FTA'].astype(str)
    game_log['OPP FTM/A'] = game_log['OPP FTM'].astype(str) + "-" + game_log['OPP FTA'].astype(str)
    game_log['OPP REB'] = game_log['REB'].astype(int)
    game_log['REB'] = game_log['reb'].astype(int)
    game_log['ORB%'] = 100 * round(game_log['off'].astype(int) / 
                              (game_log['off'].astype(int) + game_log['OPP DREB'].astype(int)), 3)
    game_log['DRB%'] = 100 * round(game_log['def'].astype(int) / 
                              (game_log['def'].astype(int) + game_log['OPP OREB'].astype(int)), 3)
    game_log['REB%'] = 100 * round(game_log['REB'].astype(int) / 
                              (game_log['REB'].astype(int) + game_log['OPP REB'].astype(int)), 3)
    game_log['TO%'] = 100 * round(game_log['to'].astype(int) / game_log['POSS'], 3)
    game_log['OPP TO%'] = 100 * round(game_log['OPP TO'].astype(int) / game_log['OPP POSS'], 3)
    game_log['POSS'] = game_log['POSS'].round(1)
    game_log['OPP POSS'] = game_log['OPP POSS'].round(1)
    
    totals_row = totals_helper(game_log)
    uaa_totals = totals_helper(game_log[(game_log.Opponent.apply(lambda o : 
                                                                 len([t for t in uaa_teams if t in o]) > 0))])
    
    game_log = game_log[['Opponent', 'W/L', 'Margin', 
                         'POSS', 'PPP', 'OPP PPP',
                         'FGM/A', '3FG/A', 'FTM/A', 'OPP FGM/A', 'OPP 3PM/A', 'OPP FTM/A',
                         'REB', 'OPP REB', 'ORB%', 'DRB%', 'REB%', 
                         'TO%', 'OPP TO%']]
    
    game_log.loc[len(game_log)] = uaa_totals
    game_log.loc[len(game_log)] = totals_row
    game_log.iloc[-2, game_log.columns.get_loc('Opponent')] = "UAA Total"
    game_log['Team'] = school

    return game_log

In [6]:
def html_df(driver, soup, school):
    events = soup.find_all("div", {"class" : "event-row"})

    results = [e.find("div", {"class" : "result"}).text.strip() for e in events]
    results = [r for r in results if r != ""]

    opponents = [e.find("div", {"class" : "opponent"}).text.strip() for e in events]
    if Team == "Women's Team":
        opponents = [o.replace("\n", "") for o in opponents if o != "Oglethorpe"]
    opponents = opponents[0:len(results)]

    if Team == "Women's Team":
        game_links = [e.find("div", {"class" : "links"}).find_all("a")[1]['href'] for e in [e for e in events[0:len(results)+1] if not str_list("Cancelled", [i.text for i in e])]]
    else:
        game_links = [e.find("div", {"class" : "links"}).find_all("a")[1]['href'] for e in events[0:len(results)]]
    game_links = ["https://www.emoryathletics.com/" + l for l in game_links]

    game_soups = []
    for url in game_links:
        driver.get(url)
        game_soups.append( BeautifulSoup(driver.page_source, 'html.parser') )

    team_df = pd.concat([opp_helper(s, "Emory", opp=False) for s in game_soups])
    team_df = team_df.reset_index(drop=True)
    team_df['Opponent'] = opponents
    opponent_df = pd.concat([opp_helper(s, "Emory") for s in game_soups])
    opponent_df = opponent_df.reset_index(drop=True)
    opponent_df['Opponent'] = opponents

    game_log = team_df.join(opponent_df, rsuffix=" OPP")

    game_log['W/L'] = [r.split(",")[0] for r in results]
    game_log['Margin'] = [eval(r.split(", ")[1]) for r in results]
    game_log['FGA'] = game_log['FGM-A'].apply(lambda i : int(i.split("-")[1]))
    game_log['FTM'] = game_log['FTM-A'].apply(lambda i : int(i.split("-")[0]))
    game_log['FTA'] = game_log['FTM-A'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP FGM'] = game_log['FGM-A OPP'].apply(lambda i : int(i.split("-")[0]))
    game_log['OPP FGA'] = game_log['FGM-A OPP'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP 3PM'] = game_log['3PM-A OPP'].apply(lambda i : int(i.split("-")[0]))
    game_log['OPP 3PA'] = game_log['3PM-A OPP'].apply(lambda i : int(i.split("-")[1]))
    game_log['OPP FGM/A'] = game_log['OPP FGM'].astype(str) + "-" + game_log['OPP FGA'].astype(str)
    game_log['OPP 3PM/A'] = game_log['OPP 3PM'].astype(str) + "-" + game_log['OPP 3PA'].astype(str)
    game_log['OPP FTM'] = game_log['FTM-A OPP'].apply(lambda i : int(i.split("-")[0]))
    game_log['OPP FTA'] = game_log['FTM-A OPP'].apply(lambda i : int(i.split("-")[1]))
    game_log['FTM/A'] = game_log['FTM'].astype(str) + "-" + game_log['FTA'].astype(str)
    game_log['OPP FTM/A'] = game_log['OPP FTM'].astype(str) + "-" + game_log['OPP FTA'].astype(str)
    game_log['OPP OREB'] = game_log['OREB OPP']
    game_log['OPP DREB'] = game_log['DREB OPP']
    game_log['OPP TO'] = game_log['TO OPP']
    game_log['POSS'] = 0.96*(game_log['FGA'] + 
                            game_log['TO'].astype(int) + 
                            0.44*game_log['FTA'] - game_log['OREB'].astype(int))
    game_log['OPP POSS'] = 0.96*(game_log['OPP FGA'] + 
                                 game_log['TO OPP'].astype(int) + 
                                 0.44*game_log['OPP FTA'] - game_log['OREB OPP'].astype(int))
    game_log['OPP PTS'] = game_log['PTS OPP'].astype(int)
    game_log['PPP'] = round(game_log['PTS'].astype(int) / game_log['POSS'], 2)
    game_log['OPP PPP'] = round(game_log['PTS OPP'].astype(int) / game_log['OPP POSS'], 2)
    game_log['FGM/A'] = game_log['FGM-A']
    game_log['3FG/A'] = game_log['3PM-A']
    game_log['OPP REB'] = game_log['REB OPP'].astype(int)
    game_log['ORB%'] = 100 * round(game_log['OREB'].astype(int) / 
                              (game_log['OREB'].astype(int) + game_log['DREB OPP'].astype(int)), 3)
    game_log['DRB%'] = 100 * round(game_log['DREB'].astype(int) / 
                              (game_log['DREB'].astype(int) + game_log['OREB OPP'].astype(int)), 3)
    game_log['REB%'] = 100 * round(game_log['REB'].astype(int) / 
                              (game_log['REB'].astype(int) + game_log['OPP REB'].astype(int)), 3)
    game_log['TO%'] = 100 * round(game_log['TO'].astype(int) / game_log['POSS'], 3)
    game_log['OPP TO%'] = 100 * round(game_log['TO OPP'].astype(int) / game_log['OPP POSS'], 3)
    game_log['POSS'] = game_log['POSS'].round(1)
    game_log['OPP POSS'] = game_log['OPP POSS'].round(1)
    
    totals_row = totals_helper(game_log)
    uaa_totals = totals_helper(game_log[(game_log.Opponent.apply(lambda o : 
                                                                 len([t for t in uaa_teams if t in o]) > 0))])
    
    game_log = game_log[['Opponent', 'W/L', 'Margin', 
                         'POSS', 'PPP', 'OPP PPP',
                         'FGM/A', '3FG/A', 'FTM/A', 'OPP FGM/A', 'OPP 3PM/A', 'OPP FTM/A',
                         'REB', 'OPP REB', 'ORB%', 'DRB%', 'REB%', 
                         'TO%', 'OPP TO%']]
    
    game_log.loc[len(game_log)] = uaa_totals
    game_log.loc[len(game_log)] = totals_row
    game_log.iloc[-2, game_log.columns.get_loc('Opponent')] = "UAA Total"
    game_log['Team'] = school

    return game_log

In [7]:
def margin_helper(df):
    def margin_flip(margin, outcome):
        if outcome == "L":
            return -abs(margin)
        else:
            return abs(margin)
    
    df['Margin'] = df.apply(lambda row: margin_flip(row['Margin'], row['W/L']), axis=1)
    
    return df

def scout_scraper(team):
    global Team
    global urls
    
    Team = team
    
    if team == "Women's Team":
        urls = wbb_urls
    else:
        urls = mbb_urls
    
    d = webdriver.Chrome(ChromeDriverManager().install())

    soups = []
    for url in list(urls.values()):
        d.get(url)
        soups.append( BeautifulSoup(d.page_source, 'html.parser') )
        
    game_logs = {"NYU" : game_by_game_df(d, soups[0], "NYU"),
                 "Rochester" : game_by_game_df(d, soups[6], "Rochester"),
                 "Emory" : html_df(d, soups[5], "Emory"),
                 "WashU" : game_by_game_df(d, soups[1], "WashU"),
                 "Brandeis" : game_log_df(d, soups[2], "Brandeis"),
                 "Case Western" : game_log_df(d, soups[3], "Case Western"),
                 "Carnegie Mellon" : game_log_df(d, soups[4], "Carnegie Mellon")}

    d.quit()
    
    return game_logs

In [8]:
m_game_logs = scout_scraper("Men's Team")
m_game_logs = margin_helper( pd.concat(list(m_game_logs.values())) ).replace(np.nan, "")
m_game_logs.to_csv("mbb_uaa_scout.csv")

w_game_logs = scout_scraper("Women's Team")
w_game_logs = margin_helper( pd.concat(list(w_game_logs.values())) ).replace(np.nan, "")
w_game_logs.to_csv("wbb_uaa_scout.csv")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 109.0.5414
[WDM] - Get LATEST chromedriver version for 109.0.5414 google-chrome
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache
/var/folders/91/7lxj2xq13852sql471wdl0pw0000gn/T/ipykernel_26162/3951410143.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome(ChromeDriverManager().install())


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 109.0.5414
[WDM] - Get LATEST chromedriver version for 109.0.5414 google-chrome
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache
/var/folders/91/7lxj2xq13852sql471wdl0pw0000gn/T/ipykernel_26162/3951410143.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome(ChromeDriverManager().install())


In [121]:
d = webdriver.Chrome(ChromeDriverManager().install())

soup = soups[5]

events = soup.find_all("div", {"class" : "event-row"})

results = [e.find("div", {"class" : "result"}).text.strip() for e in events]
results = [r for r in results if r != ""]

opponents = [e.find("div", {"class" : "opponent"}).text.strip() for e in events]
opponents = [o.replace("\n", "") for o in opponents if o != "Oglethorpe"]
opponents = opponents[0:len(results)]

game_links = [e.find("div", {"class" : "links"}).find_all("a")[1]['href'] for e in [e for e in events[0:len(results)+1] if not str_list("Cancelled", [i.text for i in e])]]
game_links = ["https://www.emoryathletics.com/" + l + "?view=plays" for l in game_links]
game_links = game_links[-6:]

game_soups = []
for url in game_links:
    d.get(url)
    game_soups.append( BeautifulSoup(d.page_source, 'html.parser') )
    
game_soups = game_soups[0:3] + game_soups[4:6]

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 109.0.5414
[WDM] - Get LATEST chromedriver version for 109.0.5414 google-chrome
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache
/var/folders/91/7lxj2xq13852sql471wdl0pw0000gn/T/ipykernel_29130/3875494743.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome(ChromeDriverManager().install())


In [111]:
def time_passed(start, end):
    start_time = int(start.split(":")[0])*60 + int(start.split(":")[1])
    end_time = int(end.split(":")[0])*60 + int(end.split(":")[1])
    
    if start_time > end_time:
        return start_time - end_time
    else:
        return 0

In [321]:
game = game_soups[4]
home = True

rows = game.find("div", {"class" : "stats-box full"}).find_all("tr")
times = [r.find("td", {"class" : "time"}).text for r in rows if (r.find("td", {"class" : "time"}) != None)]
away_plays = [r.find_all("td", {"class" : "play"})[0].text.strip() for r in rows if (r.find("td", {"class" : "time"}) != None)]
home_plays = [r.find_all("td", {"class" : "play"})[1].text.strip() for r in rows if (r.find("td", {"class" : "time"}) != None)]

plays = []
teams = []
for x in range(0, len(away_plays)):
    if away_plays[x] == "":
        plays.append(home_plays[x])
        teams.append("Opponent")
    else:
        plays.append(away_plays[x])
        teams.append("Emory")

emory = pd.DataFrame({'Time': times,
                      'Play': plays,
                      'Team': teams})

emory.to_csv("emory_brandeis.csv")


In [228]:
def new_quarter_check(start, end):
    first = int(start.split(":")[0])*60 + int(start.split(":")[1])
    last = int(end.split(":")[0])*60 + int(end.split(":")[1])
    
    return (first < last)

def and_one(df):
    return ("made" in df.Action.values[0]) and ("Foul" in df.Action.values[1]) and ("free throw" in df.Action.values[2])

def check_oreb(df):
    for x in df.values:
        if "offensive rebound" in x[1]:
            if "Emory" == x[2]:
                return True

In [378]:
def possession_length(df):
    possessions = []
    current = 1
    for x in range(0, len(df)):
        indexer = df.shape[0] - x - 1
        if ('missed' in df.Action.values[indexer]) | ('made' in df.Action.values[indexer]) | ('Turnover' in df.Action.values[indexer]):
            possessions.append(current)
            current += 1
        else:
            possessions.append(current)

    df['Possession'] = possessions[::-1]

    lengths = []
    outcomes = []
    for x in pd.unique(df.Possession):
        new_df = pd.concat([df[(df.Possession == x)],
                            df[(df.index == 1 + df[(df.Possession == x)].index.values[-1])] ])

        new_df = new_df[(~new_df.Action.str.contains("goes to the bench")) & (~new_df.Action.str.contains("enters the game")) & (~new_df.Action.str.contains("TIMEOUT"))]

        if "free throw" in new_df.Action.values[-1]:
            outcome = "Shooting Foul"
        elif "missed" in new_df.Action.values[-1]:
            outcome = "Miss"
        elif "made" in new_df.Action.values[-1]:
            outcome = "Make"
        elif "Turnover" in new_df.Action.values[-1]:
            outcome = "Turnover"
        else:
            outcome = "FLAG"

        start = new_df.Time.values[0]
        if len(new_df) == 1:
            continue
        if new_quarter_check(new_df.Time.values[0], new_df.Time.values[1]):
            start == "10:00"
        if new_df.Time.values[0] == "00:21":
            if new_df.Action.values[0] == "ARONSKY,DANIELLA made 3-pt. jump shot":
                start = "10:00"
        if new_df.Time.values[0] == "00:00":
            if new_df.Action.values[0] == "OR,CATHERINE missed 3-pt. jump shot":
                start = "10:00"
        if new_df.Time.values[0] == "00:01":
            if new_df.Action.values[0] == "MARTIN,ERIN missed 3-pt. jump shot":
                start = "10:00"
        if new_df.Time.values[0] == "00:03":
            if new_df.Action.values[0] == "ARONSKY,DANIELLA missed 3-pt. jump shot":
                start = "10:00"
        if new_df.Time.values[0] == "00:02":
            if new_df.Action.values[0] == "GROSS,PAIGE missed layup":
                start = "10:00"
        if new_df.Time.values[0] == "00:00":
            if new_df.Action.values[0] == "Turnover by ARONSKY,DANIELLA":
                start = "10:00"
        if new_df.Time.values[0] == "00:01":
            if new_df.Action.values[0] == "Turnover by ARONSKY,DANIELLA":
                start = "10:00"
        if new_df.Time.values[0] == "00:00":
            if new_df.Action.values[0] == "PELLECCHIA,BELLE missed layup":
                start = "10:00"
        if new_df.Time.values[0] == "00:00":
            if new_df.Action.values[0] == "BRUNS,NATALIE missed jump shot":
                start = "10:00"
        if new_df.Time.values[0] == "00:00":
            if new_df.Action.values[0] == "BRUNS,NATALIE missed jump shot":
                start = "10:00"
        
        if new_df.Time.values[-1] == '--':
            end = new_df.Time.values[-2]
        else:
            end = new_df.Time.values[-1]
            
        if new_df.Team.values[-1] == 'Emory':
            new_df['new'] = new_df.Action.apply(lambda i : "free throw" in i)
            
            if and_one(new_df):
                continue
            
       #     if check_oreb(new_df):
        #        print(time_passed(start, end))
        #        display(new_df)
            
            if (new_df.new.sum() <= 1) and (outcome != "FLAG"):
                lengths.append(time_passed(start, end))
                outcomes.append(outcome)
                
              #  if time_passed(start, end) == 0:
                  #  print(time_passed(start, end))
                 #   display(new_df)
            
    return pd.DataFrame({'Length' : lengths, 'Outcome' : outcomes})


In [379]:
new_emory = pd.read_csv("emory_brandeis.csv").iloc[:, 1:]

possession_length(new_emory)#.to_csv("emory_brandeis_possessions.csv")

19


,Time,Action,Team,Possession,new
3,09:23,"ARONSKY,DANIELLA missed 3-pt. jump shot",Emory,183,False
4,09:23,TEAM offensive rebound,Emory,183,False
5,09:04,"Turnover by ARONSKY,DANIELLA",Emory,182,False


4


,Time,Action,Team,Possession,new
50,05:23,"BROCK,CLAIRE missed jump shot",Emory,164,False
51,05:23,"GROSS,PAIGE offensive rebound",Emory,164,False
52,05:19,"BROCK,CLAIRE missed 3-pt. jump shot",Emory,163,False


4


...

7


,Time,Action,Team,Possession,new
465,02:47,"MARTINI,KATHERINE missed jump shot",Emory,15,False
466,02:47,"KREUSSER,CHLOE offensive rebound",Emory,15,False
467,02:40,"Foul by GONZALEZ,SELENYA",Opponent,15,False
468,02:40,"KENNEDY,LILY made free throw",Emory,14,True


20


,Time,Action,Team,Possession,new
476,02:36,"LOWNDES,CHARLOTTE missed layup",Emory,10,False
477,02:36,TEAM offensive rebound,Emory,10,False
478,02:16,"MARTINI,KATHERINE made jump shot",Emory,9,False


,Length,Outcome
0,4,Miss
1,19,Turnover
2,19,Miss
3,22,Make
4,7,Make
5,2,Turnover
6,11,Make
7,8,Miss
8,7,Make
9,6,Miss


In [355]:
emory_possessions = pd.concat([pd.read_csv("emory_brandeis_possessions.csv"),
                               pd.read_csv("emory_nyu_possessions.csv"),
                               pd.read_csv("emory_carnegie_possessions.csv"),
                               pd.read_csv("emory_rochester_possessions.csv"),
                               pd.read_csv("emory_case_possessions.csv")]).iloc[:, 1:]

In [362]:
emory_possessions['<15 seconds'] = emory_possessions['Length'] < 15
emory_possessions['15+ seconds'] = emory_possessions['Length'] >= 15

poss15 = [sum(emory_possessions['<15 seconds']), sum(emory_possessions['15+ seconds'])]
freq15 = [round(100*(p / sum(poss15)), 1) for p in poss15]

poss_length = pd.DataFrame({"Possession Length" : ['<15 seconds', '15+ seconds'],
                            "Count" : poss15,
                            "Frequency" : freq15})

poss_length = poss_length.set_index("Possession Length")

In [364]:
print("Average Possession Length (in 5 UAA games):", np.mean(emory_possessions.Length).round(1), "sec")
display(poss_length)

Average Possession Length (in 5 UAA games): 13.7 sec


,Count,Frequency
Possession Length,,
<15 seconds,251,60.9
15+ seconds,161,39.1
